<h1><center> Trabajo Práctico 3: Fashion detector </center></h1>

In [ ]:

# lib para trabajar con arrays
import numpy as np

# lib que usamos para mostrar las imágenes
import matplotlib.pyplot as plt

# libs que usamos para construir y entrenar redes neuronales, y que además tiene utilidades para leer sets de 
# imágenes
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Input, Dropout, Convolution2D, MaxPooling2D, Flatten
from tensorflow.keras.preprocessing.image import load_img, img_to_array, ImageDataGenerator

# libs que usamos para tareas generales de machine learning. En este caso, métricas
from sklearn.metrics import accuracy_score, confusion_matrix

# configuración para que las imágenes se vean dentro del notebook
%matplotlib inline
import tensorflow as tf
from tensorflow import keras

In [ ]:
fashion_mnist = keras.datasets.fashion_mnist
# esto es para que los calores de los modelos den siempre igual
np.random.seed(2)
tf.random.set_seed(2)

(x_train, x_lab), (y_validation, y_lab) = fashion_mnist.load_data()

Clases:

| Label | Clase |
| -- | --- |
| 0 | T-shirt/top |
| 1 | Trouser |
| 2 | Pullover|
| 3 | Dress |
| 4 | Coat |
| 5 | Sandal |
| 6 | Shirt |
| 7 | Sneaker |
| 8 | Bag |
| 9 | Ankle boot |


In [ ]:
clases = "camiseta/top","Pantalón","Pulover","Vestido","Saco","Sandalia","Camisa","Zapatilla","Bolso","Botas"


In [ ]:
#Exploración de datos:
x_train.shape
#Tiene 60000 imagenes de 28x28 pixeles.

In [ ]:
#Contiene etiquetas del 0 al 9.
x_lab

In [ ]:
y_validation.shape
#TIene 10000 imagenes.

In [ ]:
#Los valores de la imagen se encuentran entre 0 y 255, por lo cual es necesario normalizar.
x_train_m = keras.utils.normalize(x_train, axis = 1)


In [ ]:
def sample_images(images,lab):
    plt.figure(figsize = (10,10))
    
    for i in range(25):
        plt.subplot(5,5,i+1)
        plt.xticks([])
        plt.yticks([])
        plt.grid("off")
        plt.imshow(images[i])
        plt.xlabel(clases[lab[i]])
        

In [ ]:
#Train sin normalizar
sample_images(x_train,x_lab)

In [ ]:
#Train normalizado.:
sample_images(x_train_m,x_lab)

In [ ]:
#Validation sin normalizar
sample_images(y_validation,y_lab)

In [ ]:
#Definiendo modelo
model = Sequential()
model.add(Flatten(input_shape=((28,28))))
#Capa oculta con 200 neuronas
model.add(Dense(200, activation ="relu"))
model.add(Dense(10, activation ="softmax"))

#compilar el modelo
model.compile(optimizer = "adam", loss = "sparse_categorical_crossentropy",metrics = ["accuracy"])
model.fit(x_train,x_lab,epochs = 20)


In [ ]:
model.evaluate(y_validation,y_lab)

In [ ]:
p = model.predict(y_validation[:10])
print(p)

In [ ]:
pred = np.argmax(p, axis=1)
print(pred)
print(y_lab[:10])

In [ ]:
for i in range(10):
    plt.imshow(y_validation[i], cmap="binary")
    plt.title("Original: {}, Predicted: {}".format(y_lab[i],pred[i]))
    plt.axis("Off")
    plt.figure()

<h3><center>red convolucional</center></h3>

In [ ]:
model_rc = Sequential([
    # el shape de los inputs es alto_imagen * ancho_imagen * cantidad_colores
    Convolution2D(input_shape=(28, 28, 1), filters=10, kernel_size=(3, 3), strides=1, activation='relu', padding='same'),
    
    # kernels de 4x4x3, y salida de 61x61x8
    Dropout(0.2),

    Convolution2D(filters=8, kernel_size=(3, 3), strides=1, activation='relu'),
    # kernels de 4x4x8, y salida de 58x58x8
    Dropout(0.2),
    
    MaxPooling2D(pool_size=(2, 2)),
    # salida de 14x14x8
    
    Flatten(),
    # salida de 1568
    
    Dense(10, activation='tanh'),
    # salida de 10
    Dropout(0.2),

    Dense(10, activation='tanh'),
    # salida de 10
    Dropout(0.2),

    Dense(10, activation='softmax'),
    # salida de 10
])
  
model_rc.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy', #categorical_crossentropy',
    metrics=['accuracy', ], #'mse', 'acc'
)

In [ ]:
model_rc.summary()

In [ ]:
#como esta compuesto un elemento del train
x_train_m[0]

In [ ]:
#como esta compuesto un elemento del lab
x_lab[0]

In [ ]:
# esto es para que los calores de los modelos den siempre igual
np.random.seed(3)
tf.random.set_seed(3)

In [ ]:
#https://programmerclick.com/article/19151150055/
#Para convertir los datos en la memoria en Dataset
#train_dataset = tf.data.Dataset.from_tensors((x_train_m,x_lab))
train_dataset = tf.data.Dataset.from_tensor_slices((x_train_m,x_lab)).batch(64)

In [ ]:
#Para convertir los datos en la memoria en Dataset
#test_dataset = tf.data.Dataset.from_tensors((y_validation,y_lab))
test_dataset = tf.data.Dataset.from_tensor_slices((y_validation, y_lab)).batch(64)


In [ ]:
# anda 
history_rc = model_rc.fit(
    train_dataset,
    epochs=10,
    batch_size=50,
    validation_data=test_dataset,
)

In [ ]:
# no hace falta hacerlo
#model_rc.evaluate(test_dataset)

### Graficamos accuracy de ambos conjuntos durante todo el proceso

In [ ]:
# función para plotear los resultados 
def plt_history(history_fit):
    plt.plot(history_fit.history['accuracy'], label='train')
    plt.plot(history_fit.history['val_accuracy'], label='validation')
    plt.title('Accuracy sobre las épocas de train')
    plt.ylabel('Accuracy')
    plt.xlabel('épocas')
    plt.legend(loc='upper left')
    plt.show()

In [ ]:
plt_history(history_rc)

In [ ]:
# anda 
#history_rc_2 = model_rc.fit(
#    train_dataset,
#    epochs=20,
    #batch_size=50
    #validation_data=test_dataset,
#)

In [ ]:
#este esta mal
#model_rc.evaluate(test_dataset)

In [ ]:
# plt.plot(history_rc_2.history['accuracy'], label='train')
# plt.plot(history_rc_2.history['val_accuracy'], label='validation')
# plt.title('Accuracy sobre las épocas de train')
# plt.ylabel('Accuracy')
# plt.xlabel('épocas')
# plt.legend(loc='upper left')
# plt.show()

### Otra red convolucional que no para ver que no sobreentrene

In [ ]:
model_rc_2 = Sequential([
    # el shape de los inputs es alto_imagen * ancho_imagen * cantidad_colores
    Convolution2D(input_shape=(28, 28, 1), filters=7, kernel_size=(3, 3), strides=1, activation='relu', padding='same'),
    
    # kernels de 4x4x3, y salida de 61x61x8
    Dropout(0.2),

    Convolution2D(filters=6, kernel_size=(3, 3), strides=1, activation='relu'),
    # kernels de 4x4x8, y salida de 58x58x8
    Dropout(0.2),
    
    MaxPooling2D(pool_size=(2, 2)),
    # salida de 14x14x8
    
    Flatten(),
    # salida de 
    
    Dense(8, activation='tanh'),
    # salida de 8
    Dropout(0.2),

    Dense(8, activation='tanh'),
    # salida de 8
    Dropout(0.2),

    Dense(10, activation='softmax'),
    # salida de 10
])
  
model_rc_2.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy', #categorical_crossentropy',
    metrics=['accuracy', ], #'mse', 'acc'
)

In [ ]:
model_rc_2.summary()

In [ ]:
# esto es para que los calores de los modelos den siempre igual
np.random.seed(6)
tf.random.set_seed(6)

In [ ]:
history_rc_2 = model_rc_2.fit(
    train_dataset,
    epochs=10,
    batch_size=50,
    validation_data=test_dataset,
)

In [ ]:
plt_history(history_rc_2)